In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install emoji google_trans_new thulac gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 MB 9.5 MB/s eta 0:00:00


In [ ]:
import nltk
import pandas as pd
import csv
import re
import emoji
import pandas as pd
import numpy as np

control_file = 'control_comments_male.txt'
depressed_file = 'depressed_comments_male.txt'

with open(depressed_file, "r", encoding="utf-8") as d:
    d_lines = d.readlines()
    d_lines = [x.replace("\n","") for x in d_lines]

with open(control_file, "r", encoding="utf-8") as c:
    c_lines = c.readlines()
    c_lines = [x.replace("\n","") for x in c_lines]

depressed = pd.DataFrame(d_lines,columns=["Tweet"])
depressed["Depressed"] = 1
control = pd.DataFrame(c_lines,columns=["Tweet"])
control["Depressed"] = 0

df = pd.concat([depressed, control], ignore_index=True)
df = df[df['Tweet'] != '']
df = df[df['Tweet'] != '转发微博']
df = df.sample(frac = 1)
df.reset_index(drop=True, inplace=True)
df.head()



,Tweet,Depressed
0,#正能量#【郑州一饭店请老人免费吃饭 还能打包带走】10月17日上午11点许，尽管郑州市区下...,0
1,如果不爱我，请一定要告诉我，我好把自己收拾利落然后离开拜托🙏，请不要伤害我，更不要骗我,1
2,我围观了@叶璇 的回答，该问题价值44.00元，围观仅1元，快来一起围观~ 微博问答,1
3,战狼★★★★，挺辛苦的。。为了中国电影也满拼的。 五棉线,0
4,没有好好告别的人还会再遇见吧,0


In [ ]:
print("Dataset size:",df.shape)
df.groupby("Depressed").count()

Dataset size: (532539, 2)


,Tweet
Depressed,
0,394006
1,138533


In [ ]:
from google_trans_new import google_translator as Translator
from joblib import Parallel, delayed
import thulac #https://github.com/thunlp/THULAC-Python

translator = Translator()
chinese_tokenizer = thulac.thulac(seg_only=True)

def remove_noice(tweet):
     tweet = re.sub(r"[^\u4e00-\u9fff]","", tweet.strip())
     return tweet.replace(" ","")

def replace_emoji(tweet):
  emoji_list = emoji.distinct_emoji_list(tweet)

  def translate_emoji(emoji_icon):
    first_token = next(emoji.analyze(emoji_icon))
    return first_token.value.data['zh'] if first_token else ""

  for emoji_icon in emoji_list:
    tweet = tweet.replace(emoji_icon, translate_emoji(emoji_icon))
  return tweet




df["emojilessTweet"] = df["Tweet"].apply(lambda x: replace_emoji(x))
df["noiselessTweet"] = df["emojilessTweet"].apply(lambda x: remove_noice(x))



Model loaded succeed


In [ ]:
df.head(50)

,Tweet,Depressed,emojilessTweet,noiselessTweet
0,#正能量#【郑州一饭店请老人免费吃饭 还能打包带走】10月17日上午11点许，尽管郑州市区下...,0,#正能量#【郑州一饭店请老人免费吃饭 还能打包带走】10月17日上午11点许，尽管郑州市区下...,正能量郑州一饭店请老人免费吃饭还能打包带走月日上午点许尽管郑州市区下起了小雨但在郑州工人路与...
1,如果不爱我，请一定要告诉我，我好把自己收拾利落然后离开拜托🙏，请不要伤害我，更不要骗我,1,如果不爱我，请一定要告诉我，我好把自己收拾利落然后离开拜托:双手合十:，请不要伤害我，更不要骗我,如果不爱我请一定要告诉我我好把自己收拾利落然后离开拜托双手合十请不要伤害我更不要骗我
2,我围观了@叶璇 的回答，该问题价值44.00元，围观仅1元，快来一起围观~ 微博问答,1,我围观了@叶璇 的回答，该问题价值44.00元，围观仅1元，快来一起围观~ 微博问答,我围观了叶璇的回答该问题价值元围观仅元快来一起围观微博问答
3,战狼★★★★，挺辛苦的。。为了中国电影也满拼的。 五棉线,0,战狼★★★★，挺辛苦的。。为了中国电影也满拼的。 五棉线,战狼挺辛苦的为了中国电影也满拼的五棉线
4,没有好好告别的人还会再遇见吧,0,没有好好告别的人还会再遇见吧,没有好好告别的人还会再遇见吧
5,抑郁症病因与发病机制还不明确，也无明显的体征和实验室指标异常，概括的说是生物、心理、社会因素...,1,抑郁症病因与发病机制还不明确，也无明显的体征和实验室指标异常，概括的说是生物、心理、社会因素...,抑郁症病因与发病机制还不明确也无明显的体征和实验室指标异常概括的说是生物心理社会因素相互作用...
6,迎泽公园尽收眼底 太原·东方明珠・湖滨广场,0,迎泽公园尽收眼底 太原·东方明珠・湖滨广场,迎泽公园尽收眼底太原东方明珠湖滨广场
7,现实与理想总是天壤之别，，，真的好讨厌那种命运被握在别人手中的感觉，为何自己不能主动出击！人...,0,现实与理想总是天壤之别，，，真的好讨厌那种命运被握在别人手中的感觉，为何自己不能主动出击！人...,现实与理想总是天壤之别真的好讨厌那种命运被握在别人手中的感觉为何自己不能主动出击人生够了
8,看书，是唯一能让我静下来的事情,1,看书，是唯一能让我静下来的事情,看书是唯一能让我静下来的事情
9,分享图片,0,分享图片,分享图片


In [ ]:
# Due to the dataset is large, a lexicon is used to filter the dataframe to extract rows that are highly relevant.

lexicon_df = pd.read_csv("depressionLexiconNew.csv", encoding="utf-8")
print(lexicon_df["context"][lexicon_df["depression_point"]==1].tolist())

['抑郁', '崩溃', '压力', '自杀', '绝食', '焦躁', '伤心', '疲惫', '死亡', '孤独', '失眠', '难受', '绝望', '放弃', '卑微', '离开', '恐惧', '危险', '封闭', '敏感', '茫然', '阴影', '摧毁', '怀疑', '崩塌', '黑暗', '无助', '愧疚', '负面', '沮丧', '紧张', '忧郁', '废物', '跳楼', '遗忘', '再见', '割腕', '边缘', '阴霾', '抗抑郁药', '不稳定的', '稳定情绪', '不稳', '企稳', '不冷不热', '不高兴地', '兴奋', '缺乏自信', '自负', '过于自信', '自卑', '怯懦', '消极', '少健康', '不齿', '真令', '虔敬', '羞愧', '深表', '痛惜', '歉疚', '太完美', '孤残', '腰腿', '肾气', '肝肾', '肾阳', '柔韧性', '补肾', '虚弱', '渐强', '更弱', '善感', '聪敏', '伪善', '慈悲心', '悲悯', '太认真', '不能同意', '劣势', '短板', '不划算', '不合算', '畸形发展', '苍绿', '清楚', '活明白', '弄懂', '搞清楚', '明白事理', '搞懂', '弄清楚', '知道', '明明白白', '搞不懂', '抑郁症', '忧郁症', '产后抑郁症', '狂躁症', '躁郁症', '愁闷', '疲乏', '抗抑郁', '胸闷', '烦躁', '易怒', '积郁', '烦躁不安', '闷闷不乐', '苦闷', '困乏', '郁结', '情绪低落', '压抑感', '心情阴郁', '神经衰弱', '烦闷', '烦燥', '毒瘾', '痴呆', '狂躁', '痉挛', '体弱', '忘事', '消沉', '消极情绪', '气闷', '心理疾病', '心绞痛', '生闷气', '低血压', '犯病', '甲亢', '阴郁', '焦虑', '嗜睡', '愤懑', '难抑', '痴呆症', '高血糖', '郁闷', '焦虑症', '疲劳感', '自闭', '胃胀', '妇科炎症', '离群', '精神紧张', '悲恸', '神经性', '痢疾', '不治之症', '压抑', '困倦', '心绪不宁

In [ ]:
depressed_lexicon_keywords = lexicon_df["context"][lexicon_df["depression_point"]==1].tolist()
pattern = re.compile('|'.join(depressed_lexicon_keywords))

filtered_depressed_df = df[df['Depressed'] == 1]
filtered_depressed_df = filtered_depressed_df[filtered_depressed_df["noiselessTweet"].astype(str).apply(lambda x: pattern.search(x) is not None)]
filtered_depressed_df.head(50)

,Tweet,Depressed,emojilessTweet,noiselessTweet
1,如果不爱我，请一定要告诉我，我好把自己收拾利落然后离开拜托🙏，请不要伤害我，更不要骗我,1,如果不爱我，请一定要告诉我，我好把自己收拾利落然后离开拜托:双手合十:，请不要伤害我，更不要骗我,如果不爱我请一定要告诉我我好把自己收拾利落然后离开拜托双手合十请不要伤害我更不要骗我
5,抑郁症病因与发病机制还不明确，也无明显的体征和实验室指标异常，概括的说是生物、心理、社会因素...,1,抑郁症病因与发病机制还不明确，也无明显的体征和实验室指标异常，概括的说是生物、心理、社会因素...,抑郁症病因与发病机制还不明确也无明显的体征和实验室指标异常概括的说是生物心理社会因素相互作用...
19,为什么他们吵完架，过一会就跟没事人一样，而我一整天都在烦，一直在想我到底在烦什么，为什么要拿...,1,为什么他们吵完架，过一会就跟没事人一样，而我一整天都在烦，一直在想我到底在烦什么，为什么要拿...,为什么他们吵完架过一会就跟没事人一样而我一整天都在烦一直在想我到底在烦什么为什么要拿他们的错...
32,《天微亮》 不知不觉，又是一夜，睡梦中，看到你的模样，不知你在哪儿，有缘是否可以再见，那时的...,1,《天微亮》 不知不觉，又是一夜，睡梦中，看到你的模样，不知你在哪儿，有缘是否可以再见，那时的...,天微亮不知不觉又是一夜睡梦中看到你的模样不知你在哪儿有缘是否可以再见那时的爱是那样模糊那样懵...
57,咖啡后，我也不是那样病态地羡慕阿斗和缸子了。抑郁症是万恶之源。#抑郁症#抑郁症,1,咖啡后，我也不是那样病态地羡慕阿斗和缸子了。抑郁症是万恶之源。#抑郁症#抑郁症,咖啡后我也不是那样病态地羡慕阿斗和缸子了抑郁症是万恶之源抑郁症抑郁症
72,住院了，真的怕自己一步一步上瘾，想活的愿因只有想活着，而想死却又无数种或许像我这样的人，所接...,1,住院了，真的怕自己一步一步上瘾，想活的愿因只有想活着，而想死却又无数种或许像我这样的人，所接...,住院了真的怕自己一步一步上瘾想活的愿因只有想活着而想死却又无数种或许像我这样的人所接触的都会...
86,焦虑的要死 我怎么办啊,1,焦虑的要死 我怎么办啊,焦虑的要死我怎么办啊
88,你就像一张白纸，那么纯洁，我不忍心去玷污，你就像一个花蕊，那么羞涩，我不忍心去触碰，你就像一...,1,你就像一张白纸，那么纯洁，我不忍心去玷污，你就像一个花蕊，那么羞涩，我不忍心去触碰，你就像一...,你就像一张白纸那么纯洁我不忍心去玷污你就像一个花蕊那么羞涩我不忍心去触碰你就像一个天使那么无...
90,莫名的心痛即使外表多么的浪荡不羁笑容满面春风十里心也似刀割裂很突然挥之不去是空虚吗是无人身旁...,1,莫名的心痛即使外表多么的浪荡不羁笑容满面春风十里心也似刀割裂很突然挥之不去是空虚吗是无人身旁...,莫名的心痛即使外表多么的浪荡不羁笑容满面春风十里心也似刀割裂很突然挥之不去是空虚吗是无人身旁...
124,深夜，一个人独自在做枯燥的工作。不是读书，不是写小说，不是看球……人生，总是常常遇到这种无聊...,1,深夜，一个人独自在做枯燥的工作。不是读书，不是写小说，不是看球……人生，总是常常遇到这种无聊...,深夜一个人独自在做枯燥的工作不是读书不是写小说不是看球人生总是常常遇到这种无聊且无助的时刻此...


In [ ]:
healthy_lexicon_keywords = lexicon_df["context"][lexicon_df["depression_point"]==0].tolist()
pattern = re.compile('|'.join(healthy_lexicon_keywords))

filtered_healthy_df = df[df['Depressed'] == 0]
filtered_healthy_df = filtered_healthy_df[filtered_healthy_df["noiselessTweet"].astype(str).apply(lambda x: pattern.search(x) is not None)]
filtered_healthy_df.head(50)

,Tweet,Depressed,emojilessTweet,noiselessTweet
0,#正能量#【郑州一饭店请老人免费吃饭 还能打包带走】10月17日上午11点许，尽管郑州市区下...,0,#正能量#【郑州一饭店请老人免费吃饭 还能打包带走】10月17日上午11点许，尽管郑州市区下...,正能量郑州一饭店请老人免费吃饭还能打包带走月日上午点许尽管郑州市区下起了小雨但在郑州工人路与...
9,分享图片,0,分享图片,分享图片
13,#一起为健康代言# 不瘦下来，你永远不会知道自己可以变多好,0,#一起为健康代言# 不瘦下来，你永远不会知道自己可以变多好,一起为健康代言不瘦下来你永远不会知道自己可以变多好
16,dfu99愿有人看透你的逞强后，给你一个最温暖的怀抱。,0,dfu99愿有人看透你的逞强后，给你一个最温暖的怀抱。,愿有人看透你的逞强后给你一个最温暖的怀抱
18,今年 要赚hey很多很多钱.！还要身体健康！！！！！,0,今年 要赚hey很多很多钱.！还要身体健康！！！！！,今年要赚很多很多钱还要身体健康
29,毕业快乐一宿舍演了一部戏老师@阿牧AMS 云南约拍写真 抄送@昆明约拍 @云南约拍- 欢迎转...,0,毕业快乐一宿舍演了一部戏老师@阿牧AMS 云南约拍写真 抄送@昆明约拍 @云南约拍- 欢迎转...,毕业快乐一宿舍演了一部戏老师阿牧云南约拍写真抄送昆明约拍云南约拍欢迎转发转发私信我送自用全套预设
30,分享我的故事,0,分享我的故事,分享我的故事
43,发表了一篇转载博文 《[转载]【一周精品博文阅读】畅听美丽传说 乐享世界之最》 - [转载...,0,发表了一篇转载博文 《[转载]【一周精品博文阅读】畅听美丽传说 乐享世界之最》 - [转载...,发表了一篇转载博文转载一周精品博文阅读畅听美丽传说乐享世界之最转载一周精品博文阅读畅听美丽传...
49,伙伴即是以各自不同的形态各自不同的元素在冲突 争议 合作中协同进化伙伴 就要有情义毕竟都是 ...,0,伙伴即是以各自不同的形态各自不同的元素在冲突 争议 合作中协同进化伙伴 就要有情义毕竟都是 ...,伙伴即是以各自不同的形态各自不同的元素在冲突争议合作中协同进化伙伴就要有情义毕竟都是江湖儿女...
68,#儿童发展#中心万巴谷9国际医院建立一个以促进#儿童#获得智商和情商发展的刺激治疗机会的儿童...,0,#儿童发展#中心万巴谷9国际医院建立一个以促进#儿童#获得智商和情商发展的刺激治疗机会的儿童...,儿童发展中心万巴谷国际医院建立一个以促进儿童获得智商和情商发展的刺激治疗机会的儿童发展中心该...


In [ ]:
### Sample dataframe after filtering with lexicon
sampling_df = pd.concat([filtered_depressed_df, filtered_healthy_df], ignore_index=True)
sampling_df = sampling_df.sample(frac = 1)
sampling_df.reset_index(drop=True, inplace=True)
sampling_df.head()


,Tweet,Depressed,emojilessTweet,noiselessTweet
0,#湖南卫视乱切镜头的原因#用眼看世界，难免一叶障目;用心观世界，万物尽收于眼底;而用一颗优等...,0,#湖南卫视乱切镜头的原因#用眼看世界，难免一叶障目;用心观世界，万物尽收于眼底;而用一颗优等...,湖南卫视乱切镜头的原因用眼看世界难免一叶障目用心观世界万物尽收于眼底而用一颗优等的心去触摸世...
1,我不明白 为什么你感到难过 脆弱 却还不愿回到我怀里,1,我不明白 为什么你感到难过 脆弱 却还不愿回到我怀里,我不明白为什么你感到难过脆弱却还不愿回到我怀里
2,怎么可能是无来由的 但可能真的也是没有道理的开始难过我觉得是下午那杯饮料太甜了导致我多巴胺透...,1,怎么可能是无来由的 但可能真的也是没有道理的开始难过我觉得是下午那杯饮料太甜了导致我多巴胺透...,怎么可能是无来由的但可能真的也是没有道理的开始难过我觉得是下午那杯饮料太甜了导致我多巴胺透支...
3,#千惠健康微课堂# 【4种美白蔬菜】1.豌豆:多吃豌豆可以祛斑驻颜；2.土豆:有关美白的蔬菜...,0,#千惠健康微课堂# 【4种美白蔬菜】1.豌豆:多吃豌豆可以祛斑驻颜；2.土豆:有关美白的蔬菜...,千惠健康微课堂种美白蔬菜豌豆多吃豌豆可以祛斑驻颜土豆有关美白的蔬菜有双豆一说双豆即土豆和豌豆...
4,勇气，生话的勇气，儿子啊，我最深深的牵挂，如果，我说如果…… 你会为我哭吗？你不会骂我...,1,勇气，生话的勇气，儿子啊，我最深深的牵挂，如果，我说如果…… 你会为我哭吗？你不会骂我...,勇气生话的勇气儿子啊我最深深的牵挂如果我说如果你会为我哭吗你不会骂我懦弱吧你不要怨恨她她是最...


In [ ]:
sampling_df.count()

Tweet             123919
Depressed         123919
emojilessTweet    123919
noiselessTweet    123919
dtype: int64

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words( 'chinese' )
stopwords = [word for word in stopwords if word not in depressed_lexicon_keywords]
stopwords = [word for word in stopwords if word not in healthy_lexicon_keywords]

def remove_stopwords(tweet):
  tweet = chinese_tokenizer.cut(tweet, text=True)
  features = tweet.split(" ")
  tweet = "".join([word for word in features if word not in stopwords])
  return tweet

sampling_df["cleanTweet"] = sampling_df["noiselessTweet"].apply(lambda x: remove_stopwords(x))
sampling_df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Tweet,Depressed,emojilessTweet,noiselessTweet,cleanTweet
0,#湖南卫视乱切镜头的原因#用眼看世界，难免一叶障目;用心观世界，万物尽收于眼底;而用一颗优等...,0,#湖南卫视乱切镜头的原因#用眼看世界，难免一叶障目;用心观世界，万物尽收于眼底;而用一颗优等...,湖南卫视乱切镜头的原因用眼看世界难免一叶障目用心观世界万物尽收于眼底而用一颗优等的心去触摸世...,湖南卫视乱切镜头原因眼看世界难免一叶障目用心观世界万物收眼底颗优等心去触摸世界人物一草一木山...
1,我不明白 为什么你感到难过 脆弱 却还不愿回到我怀里,1,我不明白 为什么你感到难过 脆弱 却还不愿回到我怀里,我不明白为什么你感到难过脆弱却还不愿回到我怀里,不明白为什感到难过脆弱却还不愿回到怀里
2,怎么可能是无来由的 但可能真的也是没有道理的开始难过我觉得是下午那杯饮料太甜了导致我多巴胺透...,1,怎么可能是无来由的 但可能真的也是没有道理的开始难过我觉得是下午那杯饮料太甜了导致我多巴胺透...,怎么可能是无来由的但可能真的也是没有道理的开始难过我觉得是下午那杯饮料太甜了导致我多巴胺透支...,无真道理难过下午杯饮料太甜导致多巴胺透支明明做三分糖还甜
3,#千惠健康微课堂# 【4种美白蔬菜】1.豌豆:多吃豌豆可以祛斑驻颜；2.土豆:有关美白的蔬菜...,0,#千惠健康微课堂# 【4种美白蔬菜】1.豌豆:多吃豌豆可以祛斑驻颜；2.土豆:有关美白的蔬菜...,千惠健康微课堂种美白蔬菜豌豆多吃豌豆可以祛斑驻颜土豆有关美白的蔬菜有双豆一说双豆即土豆和豌豆...,千惠健康微课堂种美白蔬菜豌豆吃豌豆祛斑驻颜土豆美白蔬菜双豆说双豆土豆豌豆白萝卜中医白萝卜可利...
4,勇气，生话的勇气，儿子啊，我最深深的牵挂，如果，我说如果…… 你会为我哭吗？你不会骂我...,1,勇气，生话的勇气，儿子啊，我最深深的牵挂，如果，我说如果…… 你会为我哭吗？你不会骂我...,勇气生话的勇气儿子啊我最深深的牵挂如果我说如果你会为我哭吗你不会骂我懦弱吧你不要怨恨她她是最...,勇气生话勇气儿子最深深牵挂说会哭不会骂懦弱不怨恨最爱儿子骄傲


In [ ]:
sampling_df.head(50)

,Tweet,Depressed,emojilessTweet,noiselessTweet,cleanTweet
0,#湖南卫视乱切镜头的原因#用眼看世界，难免一叶障目;用心观世界，万物尽收于眼底;而用一颗优等...,0,#湖南卫视乱切镜头的原因#用眼看世界，难免一叶障目;用心观世界，万物尽收于眼底;而用一颗优等...,湖南卫视乱切镜头的原因用眼看世界难免一叶障目用心观世界万物尽收于眼底而用一颗优等的心去触摸世...,湖南卫视乱切镜头原因眼看世界难免一叶障目用心观世界万物收眼底颗优等心去触摸世界人物一草一木山...
1,我不明白 为什么你感到难过 脆弱 却还不愿回到我怀里,1,我不明白 为什么你感到难过 脆弱 却还不愿回到我怀里,我不明白为什么你感到难过脆弱却还不愿回到我怀里,不明白为什感到难过脆弱却还不愿回到怀里
2,怎么可能是无来由的 但可能真的也是没有道理的开始难过我觉得是下午那杯饮料太甜了导致我多巴胺透...,1,怎么可能是无来由的 但可能真的也是没有道理的开始难过我觉得是下午那杯饮料太甜了导致我多巴胺透...,怎么可能是无来由的但可能真的也是没有道理的开始难过我觉得是下午那杯饮料太甜了导致我多巴胺透支...,无真道理难过下午杯饮料太甜导致多巴胺透支明明做三分糖还甜
3,#千惠健康微课堂# 【4种美白蔬菜】1.豌豆:多吃豌豆可以祛斑驻颜；2.土豆:有关美白的蔬菜...,0,#千惠健康微课堂# 【4种美白蔬菜】1.豌豆:多吃豌豆可以祛斑驻颜；2.土豆:有关美白的蔬菜...,千惠健康微课堂种美白蔬菜豌豆多吃豌豆可以祛斑驻颜土豆有关美白的蔬菜有双豆一说双豆即土豆和豌豆...,千惠健康微课堂种美白蔬菜豌豆吃豌豆祛斑驻颜土豆美白蔬菜双豆说双豆土豆豌豆白萝卜中医白萝卜可利...
4,勇气，生话的勇气，儿子啊，我最深深的牵挂，如果，我说如果…… 你会为我哭吗？你不会骂我...,1,勇气，生话的勇气，儿子啊，我最深深的牵挂，如果，我说如果…… 你会为我哭吗？你不会骂我...,勇气生话的勇气儿子啊我最深深的牵挂如果我说如果你会为我哭吗你不会骂我懦弱吧你不要怨恨她她是最...,勇气生话勇气儿子最深深牵挂说会哭不会骂懦弱不怨恨最爱儿子骄傲
5,♬我正在收听Agent 51的单曲《Aim High (Album Version)》♬网页...,0,♬我正在收听Agent 51的单曲《Aim High (Album Version)》♬网页...,我正在收听的单曲网页链接来自音乐百万正版无损音乐曲库优质而上乘的听觉享受,收听单曲网页链接音乐百万正版无损音乐曲库优质上乘听觉享受
6,我一个大男人，被我的抑郁症女朋友气哭了哈哈哈哈,1,我一个大男人，被我的抑郁症女朋友气哭了哈哈哈哈,我一个大男人被我的抑郁症女朋友气哭了哈哈哈哈,一个大男人抑郁症女朋友气哭了哈哈哈哈
7,#家居装修##装修经验谈# 墙面施工的材料要围绕自己的家居风格，预算，颜色搭配质感等等综合选...,0,#家居装修##装修经验谈# 墙面施工的材料要围绕自己的家居风格，预算，颜色搭配质感等等综合选...,家居装修装修经验谈墙面施工的材料要围绕自己的家居风格预算颜色搭配质感等等综合选择设计师的风格...,家居装修装修经验谈墙面施工材料围绕家居风格预算颜色搭配质感综合选择设计师风格千人千面人喜欢乳...
8,抑郁症连发个微博，你们都报警，非得闹到爸妈那里吗？非得这样逼着吗,1,抑郁症连发个微博，你们都报警，非得闹到爸妈那里吗？非得这样逼着吗,抑郁症连发个微博你们都报警非得闹到爸妈那里吗非得这样逼着吗,抑郁症发微博都报警非得闹爸妈非得逼
9,还记得以前在艳阳天上班那是真的很累在上班的时候总爱听李宗盛 给自己的歌那个时候听不懂现在听...,1,还记得以前在艳阳天上班那是真的很累在上班的时候总爱听李宗盛 给自己的歌那个时候听不懂现在听...,还记得以前在艳阳天上班那是真的很累在上班的时候总爱听李宗盛给自己的歌那个时候听不懂现在听懂了...,还记得艳阳天上班真很累上班总爱听李宗盛歌听不懂听懂说再听已曲中人听写情绪还算稳定有时候人死不...


In [ ]:
import gensim

TweetList = df["noiselessTweet"].tolist()
TweetCorpus = []
for tweet in TweetList:
  tweet = chinese_tokenizer.cut(tweet, text=True)
  TweetCorpus.append(tweet.split(" "))

model = gensim.models.Word2Vec(TweetCorpus, window=5, min_count=1, workers=4)
model.save("tweets.model")